In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from enum import Enum


In [12]:
top_similar_df = pd.read_pickle("similarity_df.pkl")
allevents = top_similar_df['event'].unique()
display(top_similar_df.head(5), top_similar_df.shape)
apollo= top_similar_df[top_similar_df['event']=='Apollo_11']
display(apollo)

,name,plot,similarity,release_date,event,genres
23141,The Founding of a Republic,In 1945 after the end of the Second Sino-Japan...,0.604864,2009,Negotiations_to_end_apartheid_in_South_Africa,"[Chinese Movies, Drama, War film, World cinema]"
39420,The Deluge,The film is set in the 17th century during the...,0.572665,1974,Negotiations_to_end_apartheid_in_South_Africa,[War film]
6928,The Founding of a Party,"During the early 20th century, China is marked...",0.572192,2011,Negotiations_to_end_apartheid_in_South_Africa,"[Drama, Historical fiction]"
39032,7 Man Army,"During the Second Sino- Japanese war 20,000 Ja...",0.564577,1976,Negotiations_to_end_apartheid_in_South_Africa,"[Chinese Movies, Martial Arts Film]"
27370,The Final Inch,The film depicts the problems still occurring ...,0.559969,2009,Negotiations_to_end_apartheid_in_South_Africa,"[Documentary, Short Film]"


(2500, 6)

,name,plot,similarity,release_date,event,genres
38440,The Dream is Alive,The movie includes scenes from numerous shuttl...,0.653769,1985,Apollo_11,"[Documentary, Short Film]"
20760,Gray Lady Down,"An aging, respected commander Paul Blanchard, ...",0.572819,1978,Apollo_11,"[Action, ActionAdventure, Adventure, Disaster]"
9956,"Restol, The Special Rescue Squad","In 2034, the future world experienced serious ...",0.560866,1999,Apollo_11,"[Action, Animation, Science Fiction]"
7964,AVP: Alien vs. Predator,"In 2004, a satellite detects a mysterious heat...",0.550346,2004,Apollo_11,"[Action, ActionAdventure, Horror, Monster movi..."
24549,Proud,The film focuses on the meritorious service of...,0.545594,2004,Apollo_11,[Drama]
...,...,...,...,...,...,...
4967,Illegal Aliens,{{Expand section}} Guided by a holographic men...,0.46709,2007,Apollo_11,"[Action, Comedy, Parody, Science Fiction, Tele..."
25203,Getting There,"The story revolves around twin sisters who, af...",0.466869,2002,Apollo_11,"[Comedy, Family Film, Teen]"
38381,Genesis II,"In 1979, NASA scientist Dylan Hunt is working...",0.466762,1973,Apollo_11,"[Apocalyptic and postapocalyptic fiction, Dyst..."
15349,The Making of Star Wars,The special was hosted by C-3PO and R2-D2.{{c...,0.465632,1977,Apollo_11,[Documentary]


In [13]:
import torch
from transformers import BertForSequenceClassification, BertTokenizer
import torch


tokenizer = BertTokenizer.from_pretrained('ProsusAI/finbert')
model = BertForSequenceClassification.from_pretrained('ProsusAI/finbert')

def calculateSentiment(text):
    ## Referenced this article : https://towardsdatascience.com/how-to-apply-transformers-to-any-length-of-text-a5601410af7f
    ## Used the https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english from Huggingface 
    
    chunksize = 512
    tokens = tokenizer.encode_plus(text, add_special_tokens=False,
                                return_tensors='pt')

    input_id_chunks = list(tokens['input_ids'][0].split(510))
    mask_chunks = list(tokens['attention_mask'][0].split(510))

    # loop through each chunk
    for i in range(len(input_id_chunks)):
        # add CLS and SEP tokens to input IDs
        input_id_chunks[i] = torch.cat([
            torch.tensor([101]), input_id_chunks[i], torch.tensor([102])
        ])
        # add attention tokens to attention mask
        mask_chunks[i] = torch.cat([
            torch.tensor([1]), mask_chunks[i], torch.tensor([1])
        ])
        # get required padding length
        pad_len = chunksize - input_id_chunks[i].shape[0]
        # check if tensor length satisfies required chunk size
        if pad_len > 0:
            # if padding length is more than 0, we must add padding
            input_id_chunks[i] = torch.cat([
                input_id_chunks[i], torch.Tensor([0] * pad_len)
            ])
            mask_chunks[i] = torch.cat([
                mask_chunks[i], torch.Tensor([0] * pad_len)
            ])

    # # check length of each tensor
    # for chunk in input_id_chunks:
    #     print(len(chunk))

    ##  Now the final step of placing our tensors back into the dictionary style format we had before.
    input_ids = torch.stack(input_id_chunks)
    attention_mask = torch.stack(mask_chunks)

    input_dict = {
        'input_ids': input_ids.long(),
        'attention_mask': attention_mask.int()
    }
    input_dict

    ## run the model
    outputs = model(**input_dict)
    probs = torch.nn.functional.softmax(outputs[0], dim=-1)
    probs = probs.mean(dim=0)
    winner = torch.argmax(probs).item()

    return (probs, winner)
        

In [14]:
display("original shape" , top_similar_df.shape) 
top_similar_df_less = top_similar_df.sort_values(by=['similarity'], ascending=False).groupby('event').head(5)


'original shape'

(2500, 6)

In [15]:
top_similar_df_less['probability'] = top_similar_df_less['plot'].apply(lambda x: calculateSentiment(x))


Token indices sequence length is longer than the specified maximum sequence length for this model (1001 > 512). Running this sequence through the model will result in indexing errors


: 

: 